In [2]:
import json
from StringIO import StringIO
import pandas as pd
import numpy as np
import datetime
from scipy.stats import ttest_ind, f_oneway 
import matplotlib.pyplot as plt

In [3]:
#Upload most recent file
listings = pd.read_csv('Datasources/listings_augmented/listings_augmented_2018-05-23_V5.csv')

/Users/pauldefusco/anaconda2/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,4,5,6,10,11,12,13,15,17,18,20,21,23,53,54,64,127,213,217) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
high_quality_amenities_list = ['has_Other_pet(s)', 'has_Elevator_in_Building', 'has_Indoor_Fireplace', 'has_Buzzer/Wireless_Intercom',
 'has_Gym', 'has_Hot_Tub', 'has_Suitable_for_Events', 'has_Cat(s)', 'has_Pets_live_on_this_property', 'has_Safety_Card','has_Smoking_Allowed',
 'has_Pool', 'has_Pets_Allowed', 'has_Wheelchair_Accessible', 'has_Dog(s)', 'has_Breakfast', 'has_Doorman', 'has_Lock_on_Bedroom_Door']

In [ ]:
mid_quality_amenities_list = ['has_Fire_Extinguisher', 'has_Cable_TV', 'has_Air_Conditioning', 'has_Hair_Dryer',
 'has_Iron', 'has_First_Aid_Kit', 'has_Laptop_Friendly_Workspace', 'has_24-Hour_Check-in']

In [ ]:
low_quality_amenities_list = ['has_Essentials', 'has_Carbon_Monoxide_Detector', 'has_Internet', 'has_Washer', 'has_Hangers',
 'has_TV', 'has_Kitchen', 'has_Family/Kid_Friendly', 'has_Shampoo', 'has_Heating', 'has_Smoke_Detector', 'has_Free_Parking_on_Premises',
 'has_Dryer', 'has_Wireless_Internet']

In [ ]:
def listings_quality_classification(listings):
    listings['amenity_level'] = ''
    
    
    for w in listings.index:
        
        for j in high_quality_amenities_list:
            a = listings[j][w]
            
            if a == 1:
                listings['amenity_level'][w] = 'high'
                break 
                
        if listings['amenity_level'][w] == 'high':
            continue
                
        for k in mid_quality_amenities_list:
            a = listings[k][w]
            
            if a == 1:
                listings['amenity_level'][w] = 'mid_level'
                break
                
        if listings['amenity_level'][w] == 'mid_level':
            continue
            
        else:
            listings['amenity_level'][w] = 'low'
            
    return listings

In [ ]:
#Ideas:
#1) Try more amenity levels
#2) Measure impact of each amenity on price, and measure susbsets of all amenities

In [6]:
listings.filter(regex='has').columns

Index([u'host_has_profile_pic', u'has_availability', u'has_Other_pet(s)',
       u'has_Essentials', u'has_Carbon_Monoxide_Detector',
       u'has_Elevator_in_Building', u'has_Indoor_Fireplace', u'has_Internet',
       u'has_Washer', u'has_Hangers', u'has_Buzzer/Wireless_Intercom',
       u'has_TV', u'has_Gym', u'has_Fire_Extinguisher', u'has_Hot_Tub',
       u'has_Dryer', u'has_Air_Conditioning', u'has_Suitable_for_Events',
       u'has_Kitchen', u'has_Family/Kid_Friendly', u'has_Shampoo',
       u'has_Heating', u'has_Hair_Dryer', u'has_Smoke_Detector', u'has_Iron',
       u'has_Lock_on_Bedroom_Door', u'has_Free_Parking_on_Premises',
       u'has_Pets_live_on_this_property', u'has_Safety_Card', u'has_Doorman',
       u'has_Pool', u'has_Pets_Allowed', u'has_Wheelchair_Accessible',
       u'has_First_Aid_Kit', u'has_Dog(s)', u'has_Wireless_Internet',
       u'has_Cable_TV', u'has_Laptop_Friendly_Workspace', u'has_Breakfast',
       u'has_Smoking_Allowed', u'has_Cat(s)', u'has_24-Hour_Che

In [7]:
from itertools import chain, combinations

#Feed only amenity features (just hte names) in this list
lst = list(listings.filter(regex='has').columns)

candidates = list(lst)
min_range = len(lst)-1

def all_subsets(ss):
    return chain(*map(lambda x: combinations(ss, x), range(min_range, len(ss)+1)))
combos = []

for subset in all_subsets(candidates):
    combos.append(subset)

In [8]:
combos

[('host_has_profile_pic',
  'has_availability',
  'has_Other_pet(s)',
  'has_Essentials',
  'has_Carbon_Monoxide_Detector',
  'has_Elevator_in_Building',
  'has_Indoor_Fireplace',
  'has_Internet',
  'has_Washer',
  'has_Hangers',
  'has_Buzzer/Wireless_Intercom',
  'has_TV',
  'has_Gym',
  'has_Fire_Extinguisher',
  'has_Hot_Tub',
  'has_Dryer',
  'has_Air_Conditioning',
  'has_Suitable_for_Events',
  'has_Kitchen',
  'has_Family/Kid_Friendly',
  'has_Shampoo',
  'has_Heating',
  'has_Hair_Dryer',
  'has_Smoke_Detector',
  'has_Iron',
  'has_Lock_on_Bedroom_Door',
  'has_Free_Parking_on_Premises',
  'has_Pets_live_on_this_property',
  'has_Safety_Card',
  'has_Doorman',
  'has_Pool',
  'has_Pets_Allowed',
  'has_Wheelchair_Accessible',
  'has_First_Aid_Kit',
  'has_Dog(s)',
  'has_Wireless_Internet',
  'has_Cable_TV',
  'has_Laptop_Friendly_Workspace',
  'has_Breakfast',
  'has_Smoking_Allowed',
  'has_Cat(s)',
  'has_24-Hour_Check-in'),
 ('host_has_profile_pic',
  'has_availability',

In [42]:
L = []

for w in listings['amenity_level'].unique():
    L.append(listings[listings['amenity_level'] == w]['price_y'])
    
f_oneway(L[0], L[1], L[2])

F_onewayResult(statistic=6.0020610567352914, pvalue=0.0024891734006616223)